# [ LG전자_DIC ] 변수선택 및 차원 축소

## 전진선택, 후방소거, 단계적 선택법

### 강의 복습
강의자료 : 변수선택 및 차원 축소, 01 전진선택, 후방소거, 단계적 선택법

- 전진 선택법 : 설명변수가 하나도 없는 모델에서부터 시작하여 가장 유의미한 변수를 하나씩 추가해 나가는 방법
- 후방 소거법 : 모든 변수를 사용하여 구축한 모델에서 유의미하지 않은 변수를 하나씩 제거해 나가는 방법
- 단계적 선택법 : 전진 선택법과 후진 소거법을 번갈아 가면서 수행하는 변수 선택 기법

### 실습 요약
1. 본 실습에서는 전진선택, 후방소거, 단계적 선택법을 통해 변수를 선택하여 차원 축소를 진행합니다.
2. 분류 과업을 위해 선형 회귀 모델과 로지스틱 모델을 통해 각 선택법에 따른 성능 차이를 확인합니다.

---

### STEP 1. 데이터 준비하기

금일 실습에서는 Malwares Classification 데이터셋을 사용합니다.
- Malware를 분류하는 것을 목적으로 하는 데이터 셋
- 총 68개의 독립 변수를 통해 종속 변수(Not Malware=0, Malware=1)를 예측해야 함

In [ ]:
# github에서 데이터 불러오기
# !git clone https://github.com/hwk0702/2024_LGE_TSVisulaization.git
# %cd 2024_LGE_TSVisulaization

In [ ]:
import numpy as np
import pandas as pd
data = pd.read_csv('./data/Malwares Classification.csv')

In [ ]:
data.head()

In [ ]:
print(data.columns)

In [ ]:
# Label 분리 및 불필요한 변수 제거
data = data.dropna(axis=0)
labels = data.loc[:,'class']
X_data = data.drop('class',axis=1)

### STEP 2. 데이터 전처리
- 무작위 추출 방식을 활용하여 전체 데이터 셋을 Train, Validation, Test로 구분함

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, labels, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
# train 데이터를 기반으로 train/test 데이터에 대하여 standard scaling 적용 (평균 0, 분산 1) 
scaler = StandardScaler()
scaler = scaler.fit(X_train)

In [ ]:
X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns, index=X_train.index)
X_val   = pd.DataFrame(scaler.transform(X_val), columns=X_val.columns, index=X_val.index)
X_test  = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

In [ ]:
print("정규화 후 평균:", np.mean(X_train.iloc[:,5]))
print("정규화 후 분산:", np.var(X_train.iloc[:,5]))

### STEP 3. 단순 모델 학습 및 평가
- 변수 선택법을 적용하지 않았을 때, 선형 회귀 모델과 로지스틱 회귀 모델을 학습하고 그 성능을 평가함

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression

In [ ]:
# 선형 회귀 학습
line_fitter = LinearRegression()
line_fitter.fit(X_train, y_train)

In [ ]:
# 로지스틱 회귀 학습
logistic_fitter = LogisticRegression(solver="lbfgs")
logistic_fitter.fit(X_train, y_train)

In [ ]:
#Accuracy 측정
def Acc(pred,label,pri=True):
    if pri:
        print("Accuracy:",np.round(np.sum(pred==label)/len(label)*100,1),"%")
    return np.sum(pred==label)/len(label)

In [ ]:
#선형 회귀 모델 예측 결과
line_pred_test = line_fitter.predict(X_test)
#로지스틱 회귀 모델 예측 결과
logistic_pred_test = logistic_fitter.predict(X_test)

In [ ]:
#선형 회귀 성능 평가
print("선형 회귀 모델")
linear_all = Acc(line_pred_test>0.5,y_test)
print("로지스틱 회귀 모델")
logistic_all = Acc(logistic_pred_test,y_test)

In [ ]:
#logistic 모델 내장 함수 사용
print(logistic_fitter.score(X_train, y_train))
print(logistic_fitter.score(X_test, y_test))

### STEP 4. 전진 선택법
- 변수가 하나도 없는 상태로부터 시작해서 1개씩 변수를 추가하는 기법
- 적합도 함수를 활용해 해당 함수 값이 가장 높은 변수를 선택하는 방식
- Stopping Criteria를 통해 변수 선택을 중단함
    - 변수를 추가하여도 더 이상 성능이 증가하지 않음
    - 성능이 기준 이상을 만족
    - 변수의 개수가 일정 개수를 초과

<img src="./image/FowardSelection.png" width="700">

In [ ]:
def variable_selection(model, X_train, y_train, X_val, y_val, direction, threshold=0.5):
    var = list(range(len(X_train.columns)))
    current_score, best_new_score = 0.0, 0.0
    if direction == "forward":
        inmodel  =[]    # 선택된 변수가 담길 리스트
        outmodel = var    # 그 외 변수들
        while outmodel and current_score==best_new_score:
            score_candidate = []    # 점수 후보군
            for candidate in outmodel:
                temp_var = inmodel + [candidate]
                Temp_X_train = X_train.iloc[:, temp_var]
                Temp_X_val   = X_val.iloc[:, temp_var]
                model.fit(Temp_X_train,y_train)
                prediction = model.predict(Temp_X_val)
                score = Acc(prediction>=threshold, y_val,pri=False)
                score_candidate.append((score, candidate))    # 점수와 선택된 변수들 후보
              
            score_candidate.sort()    # 가장 좋은 성능 기준으로 정렬
            best_new_score, best_candidate = score_candidate.pop()
            if current_score>=best_new_score:
            #if current_score>=best_new_score and len(inmodel)>=15:
            #if current_score>=best_new_score and best_new_score>=0.943:
            #if current_score>=best_new_score or len(inmodel)==5:
                print("Finish:", inmodel)
                break
            else:
                outmodel.remove(best_candidate)
                inmodel.append(best_candidate)
                current_score = best_new_score
            
            print(inmodel)
            
        Temp_X_train = X_train.iloc[:, inmodel]
        Temp_X_val   = X_val.iloc[:, inmodel]
        model.fit(Temp_X_train, y_train)
        prediction = model.predict(Temp_X_val)
        score = Acc(prediction>=threshold, y_val, pri=False)
        print("Best Validation Acc:", round(score*100,1), "%")
        print("선택된 변수의 개수:", len(inmodel), "개")
        
        return model, inmodel

In [ ]:
#선형 회귀 모델에 대한 전진 선택법 적용 결과
model, inmodel = variable_selection(LinearRegression(), X_train, y_train, X_val, y_val, direction="forward")

In [ ]:
#선형 회귀 모델 전진 선택법 적용시 평가 데이터 셋에 대한 결과 확인
pred_test = model.predict(X_test.iloc[:, inmodel])
print("선형 회귀 모델-전진 선택법")
linear_forward = Acc(pred_test>0.5, y_test)
print("선택된 변수 개수:", len(inmodel))
print("Acc 증감:", round((linear_forward-linear_all)*100,1), "%")

In [ ]:
#로지스틱 회귀 모델에 대한 전진 선택법 적용 결과
model, inmodel = variable_selection(LogisticRegression(solver="lbfgs"), 
                                    X_train, y_train, X_val, y_val, direction="forward")

In [ ]:
#로지스틱 회귀 모델 전진 선택법 적용시 평가 데이터 셋에 대한 결과 확인
pred_test = model.predict(X_test.iloc[:, inmodel])
print("로지스틱 회귀 모델-전진 선택법")
logistic_forward = Acc(pred_test, y_test)
print("선택된 변수 개수:", len(inmodel))
print("Acc 증감:", round((logistic_forward-logistic_all)*100, 1), "%")

### STEP 5. 후진 소거법
- 전체 변수로부터 시작해서 하나씩 변수를 제거하는 방식
- 적합도 함수를 활용해 해당 함수 값이 가장 높은 변수를 제거하는 방식
- Stopping Criteria를 통해 변수 선택을 중단함
    - 변수를 제거하여도 더 이상 증가하지 않음
    - 변수의 개수가 일정 개수 이하여야 stop 가능

<img src="./image/BackwardElimination.png" width="700">

In [ ]:
def variable_selection(model, X_train, y_train, X_val, y_val, direction, threshold=0.5):
    var = list(range(len(X_train.columns)))
    current_score, best_new_score = 0.0, 0.0
    if direction == "backward":
        inmodel  = var    # 선택되지 않은 변수들
        outmodel = []    # 선택된 변수들
        while inmodel and current_score==best_new_score:
            score_candidate = []    # 점수 후보군
            for candidate in inmodel:
                temp_var = inmodel.copy()
                temp_var.remove(candidate)
                Temp_X_train = X_train.iloc[:, temp_var]
                Temp_X_val   = X_val.iloc[:, temp_var]
                model.fit(Temp_X_train,y_train)
                prediction = model.predict(Temp_X_val)
                score = Acc(prediction>=threshold, y_val,pri=False)
                score_candidate.append((score, candidate))    # 점수와 선택된 변수들 후보
              
            score_candidate.sort()    # 가장 좋은 성능 기준으로 정렬
            best_new_score, best_candidate = score_candidate.pop()
            
            if current_score>=best_new_score:
            #if current_score>=best_new_score and len(inmodel)>=15:
            #if current_score>=best_new_score and best_new_score>=0.943:
            #if current_score>=best_new_score or len(inmodel)==5:
                print("Finish:", inmodel)
                break
            else:
                outmodel.append(best_candidate)
                inmodel.remove(best_candidate)
                current_score = best_new_score
                
            print(inmodel)
            
        Temp_X_train = X_train.iloc[:, inmodel]
        Temp_X_val   = X_val.iloc[:, inmodel]
        model.fit(Temp_X_train, y_train)
        prediction = model.predict(Temp_X_val)
        score = Acc(prediction>=threshold, y_val, pri=False)
        print("Best Validation Acc:", round(score*100, 1), "%")
        print("선택된 변수의 개수:", len(inmodel), "개")
        
        return model, inmodel

In [ ]:
#선형 회귀 모델에 대한 후진 소거법 적용 결과
model, inmodel = variable_selection(LinearRegression(), X_train, y_train, X_val, y_val, direction="backward")

In [ ]:
#선형 회귀 모델 후진 소거법 적용시 평가 데이터 셋에 대한 결과 확인
pred_test = model.predict(X_test.iloc[:, inmodel])
print("선형 회귀 모델-후진 소거법")
linear_backward = Acc(pred_test>0.5, y_test)
print("선택된 변수 개수:", len(inmodel))
print("Acc 증감:", round((linear_backward-linear_all)*100, 1), "%")

In [ ]:
#로지스틱 회귀 모델에 대한 전진 선택법 적용 결과
model, inmodel = variable_selection(LogisticRegression(solver="lbfgs",max_iter=300), 
                                    X_train, y_train, X_val, y_val, direction="backward")

In [ ]:
#로지스틱 회귀 모델 후진 소거법 적용시 평가 데이터 셋에 대한 결과 확인
pred_test = model.predict(X_test.iloc[:, inmodel])
print("로지스틱 회귀 모델-후진 소거법")
logistic_backward = Acc(pred_test, y_test)
print("선택된 변수 개수:", len(inmodel))
print("Acc 증감:", round((logistic_backward-logistic_all)*100, 1), "%")

### STEP 6. 단계적 선택법
- 전진 선택법과 마찬가지로 변수가 하나도 없는 상태에서 시작
- 전진 선택법과 후진 선택법을 모두 적용하여 적합도 함수를 계산함
- 적합도 함수가 가장 높은 경우로 변수 선택 or 제거를 진행함
- Stopping Criteria를 통해 변수 선택을 중단함
    - 변수를 추가하거나 제거하여도 더 이상 성능이 증가하지 않음
    - 성능이 기준 이상을 만족
    - 변수의 개수가 일정 개수를 초과

<img src="./image/StepwiseSelection.png" width="700">

In [ ]:
def variable_selection(model, X_train, y_train, X_val, y_val, direction, threshold=0.5):
    var = list(range(len(X_train.columns)))
    current_score, best_new_score = 0.0, 0.0
    if direction == "step":
        inmodel  =[]
        outmodel = var
        stop = 0
        count = 0
        forward = 0
        backward = 0
        while outmodel and stop<2:
            score_candidate = []
            count=count+1
            if count%2==1 or count==2:
                for candidate in outmodel:
                    temp_var = inmodel + [candidate]
                    Temp_X_train = X_train.iloc[:, temp_var]
                    Temp_X_val   = X_val.iloc[:, temp_var]
                    model.fit(Temp_X_train,y_train)
                    prediction = model.predict(Temp_X_val)
                    score = Acc(prediction>=threshold, y_val,pri=False)
                    score_candidate.append((score, candidate))
              
                score_candidate.sort()
                best_new_score, best_candidate = score_candidate.pop()
                if current_score<best_new_score:
                    print("success forward")
                    outmodel.remove(best_candidate)
                    inmodel.append(best_candidate)
                    current_score = best_new_score
                    stop=0
                    forward = forward+1
                else:
                    print("fails forward")
                    stop=stop+1
            else:
                for candidate in inmodel:
                    temp_var = inmodel.copy()
                    temp_var.remove(candidate)
                    Temp_X_train = X_train.iloc[:, temp_var]
                    Temp_X_val   = X_val.iloc[:, temp_var]
                    model.fit(Temp_X_train, y_train)
                    prediction = model.predict(Temp_X_val)
                    score = Acc(prediction>=threshold, y_val, pri=False)
                    score_candidate.append((score, candidate))
                
                score_candidate.sort()
                best_new_score, best_candidate = score_candidate.pop()
                    
                if current_score<best_new_score:
                    print("success backward")
                    outmodel.append(best_candidate)
                    inmodel.remove(best_candidate)
                    current_score = best_new_score
                    stop=0
                    backward = backward+1
                else:
                    print("fails backward")
                    stop=stop+1
                
            print(inmodel)
            
        Temp_X_train = X_train.iloc[:, inmodel]
        Temp_X_val   = X_val.iloc[:, inmodel]
        model.fit(Temp_X_train,y_train)
        prediction = model.predict(Temp_X_val)
        score = Acc(prediction>= threshold, y_val, pri=False)
        print("Best Validation Acc:", round((score)*100, 1), "%")
        print("선택된 변수의 개수:", len(inmodel), "개")
        print("success forward:", forward)
        print("success backward:", backward)
        
        return model, inmodel

In [ ]:
#선형 회귀 모델에 대한 단계적 선택법 적용 결과
model, inmodel = variable_selection(LinearRegression(), X_train, y_train, X_val, y_val, direction="step")

In [ ]:
#선형 회귀 모델 단계적 선택법 적용시 평가 데이터 셋에 대한 결과 확인
pred_test = model.predict(X_test.iloc[:, inmodel])
print("선형 회귀 모델-단계적 소거법")
linear_step = Acc(pred_test>0.5, y_test)
print("선택된 변수 개수:", len(inmodel))
print("Acc 증감:", round((linear_step-linear_all)*100, 1), "%")

In [ ]:
#로지스틱 회귀 모델에 대한 단계적 선택법 적용 결과
model, inmodel = variable_selection(LogisticRegression(solver="lbfgs", max_iter=300), 
                                    X_train, y_train, X_val, y_val, direction="step")

In [ ]:
#로지스틱 회귀 모델 단계적 선택법 적용시 평가 데이터 셋에 대한 결과 확인
pred_test = model.predict(X_test.iloc[:, inmodel])
print("로지스틱 회귀 모델-단계적 선택법")
logistic_step = Acc(pred_test, y_test)
print("선택된 변수 개수:", len(inmodel))
print("Acc 증감:", round((logistic_step-logistic_all)*100, 1), "%")

In [ ]:
def variable_selection(model, X_train, y_train, X_val, y_val, direction, threshold=0.5):
    var = list(range(len(X_train.columns)))
    current_score, best_new_score = 0.0, 0.0
    if direction == "step2":
        inmodel  = var
        outmodel = []
        stop = 0
        count = 0
        forward = 0
        backward = 0
        while inmodel and stop<2:
            score_candidate = []
            count=count+1
            if count%2==0 and count!=2:
                for candidate in outmodel:
                    temp_var = inmodel + [candidate]
                    Temp_X_train = X_train.iloc[:, temp_var]
                    Temp_X_val   = X_val.iloc[:, temp_var]
                    model.fit(Temp_X_train, y_train)
                    prediction = model.predict(Temp_X_val)
                    score = Acc(prediction>=threshold, y_val, pri=False)
                    score_candidate.append((score, candidate))
              
                score_candidate.sort()
                best_new_score, best_candidate = score_candidate.pop()
                if current_score<best_new_score:
                    print("success forward")
                    outmodel.remove(best_candidate)
                    inmodel.append(best_candidate)
                    current_score = best_new_score
                    stop=0
                    forward = forward+1
                else:
                    print("falis forward")
                    stop=stop+1
            else:
                for candidate in inmodel:
                    temp_var = inmodel.copy()
                    temp_var.remove(candidate)
                    Temp_X_train = X_train.iloc[:, temp_var]
                    Temp_X_val   = X_val.iloc[:, temp_var]
                    model.fit(Temp_X_train, y_train)
                    prediction = model.predict(Temp_X_val)
                    score = Acc(prediction>=threshold, y_val, pri=False)
                    score_candidate.append((score, candidate))
                
                score_candidate.sort()
                best_new_score, best_candidate = score_candidate.pop()
                    
                if current_score<best_new_score:
                    print("success backward")
                    outmodel.append(best_candidate)
                    inmodel.remove(best_candidate)
                    current_score = best_new_score
                    stop=0
                    backward = backward+1
                else:
                    print("fails backward")
                    stop=stop+1
                
            print(inmodel)
            
        Temp_X_train = X_train.iloc[:, inmodel]
        Temp_X_val   = X_val.iloc[:, inmodel]
        model.fit(Temp_X_train, y_train)
        prediction = model.predict(Temp_X_val)
        score = Acc(prediction>=threshold, y_val, pri=False)
        print("Best Validation Acc:", round((score)*100, 1), "%")
        print("선택된 변수의 개수:", len(inmodel), "개")
        print("success forward:", forward)
        print("success backward:", backward)
        
        return model, inmodel

In [ ]:
#선형 회귀 모델에 대한 단계적 선택법2 적용 결과
model, inmodel = variable_selection(LinearRegression(), X_train,y_train,X_val,y_val,direction="step2")

In [ ]:
#선형 회귀 모델 단계적 선택법2 적용시 평가 데이터 셋에 대한 결과 확인
pred_test = model.predict(X_test.iloc[:,inmodel])
print("선형 회귀 모델-단계적 선택법2")
linear_step2 = Acc(pred_test>0.5,y_test)
print("선택된 변수 개수:", len(inmodel))
print("Acc 증감:",round((linear_step2-linear_all),1),"%")

In [ ]:
#로지스틱 회귀 모델에 대한 전진 선택법2 적용 결과
model, inmodel = variable_selection(LogisticRegression(solver="lbfgs", max_iter=300), 
                                    X_train, y_train, X_val, y_val, direction="step2")

In [ ]:
#로지스틱 회귀 모델 단계적 선택법2 적용시 평가 데이터 셋에 대한 결과 확인
pred_test = model.predict(X_test.iloc[:, inmodel])
print("로지스틱 회귀 모델-단계적 선택법2")
logistic_step2 = Acc(pred_test, y_test)
print("선택된 변수 개수:", len(inmodel))
print("Acc 증감:", round((logistic_step2-logistic_all)*100, 1), "%")

### STEP 7. 변수 선택법 함수

In [ ]:
def variable_selection(model, X_train, y_train, X_val, y_val, direction, threshold=0.5):
    var = list(range(len(X_train.columns)))
    current_score, best_new_score = 0.0, 0.0
    ### 전진 선택법
    if direction == "forward":
        inmodel  =[]
        outmodel = var
        while outmodel and current_score==best_new_score:
            score_candidate = []
            for candidate in outmodel:
                temp_var = inmodel + [candidate]
                Temp_X_train = X_train.iloc[:, temp_var]
                Temp_X_val   = X_val.iloc[:, temp_var]
                model.fit(Temp_X_train,y_train)
                prediction = model.predict(Temp_X_val)
                score = Acc(prediction>=threshold, y_val, pri=False)
                score_candidate.append((score, candidate))
              
            score_candidate.sort()
            best_new_score, best_candidate = score_candidate.pop()
            if current_score>=best_new_score:
                print(inmodel)
                break
            else:
                outmodel.remove(best_candidate)
                inmodel.append(best_candidate)
                current_score = best_new_score
            
            print(inmodel)
            
        Temp_X_train = X_train.iloc[:, inmodel]
        Temp_X_val   = X_val.iloc[:, inmodel]
        model.fit(Temp_X_train, y_train)
        prediction = model.predict(Temp_X_val)
        score = Acc(prediction>=threshold, y_val, pri=False)
        print("Best Validation Acc:", round((score)*100, 1), "%")
        print("선택된 변수의 개수:", len(inmodel), "개")
    
    ###후진 소거법
    if direction == "backward":
        inmodel  = var
        outmodel = []
        while inmodel and current_score==best_new_score:
            score_candidate = []
            for candidate in inmodel:
                temp_var = inmodel.copy()
                temp_var.remove(candidate)
                Temp_X_train = X_train.iloc[:, temp_var]
                Temp_X_val   = X_val.iloc[:, temp_var]
                model.fit(Temp_X_train,y_train)
                prediction = model.predict(Temp_X_val)
                score = Acc(prediction>=threshold, y_val, pri=False)
                score_candidate.append((score, candidate))
              
            score_candidate.sort()
            best_new_score, best_candidate = score_candidate.pop()
            
            if current_score>=best_new_score:
                print(inmodel)
                break
            else:
                outmodel.append(best_candidate)
                inmodel.remove(best_candidate)
                current_score = best_new_score
                
            print(inmodel)
            
        Temp_X_train = X_train.iloc[:, inmodel]
        Temp_X_val   = X_val.iloc[:, inmodel]
        model.fit(Temp_X_train, y_train)
        prediction = model.predict(Temp_X_val)
        score = Acc(prediction>=threshold, y_val, pri=False)
        print("Best Validation Acc:", round((score)*100, 1), "%")
        print("선택된 변수의 개수:", len(inmodel), "개")
        
    ###단계적 선택법
    if direction == "step":
        inmodel  =[]
        outmodel = var
        stop = 0
        count = 0
        forward = 0
        backward = 0
        while outmodel and stop<2:
            score_candidate = []
            count=count+1
            if count%2==1 or count==2:
                for candidate in outmodel:
                    temp_var = inmodel + [candidate]
                    Temp_X_train = X_train.iloc[:, temp_var]
                    Temp_X_val   = X_val.iloc[:, temp_var]
                    model.fit(Temp_X_train,y_train)
                    prediction = model.predict(Temp_X_val)
                    score = Acc(prediction>=threshold, y_val, pri=False)
                    score_candidate.append((score, candidate))
              
                score_candidate.sort()
                best_new_score, best_candidate = score_candidate.pop()
                if current_score<best_new_score:
                    print("success forward")
                    outmodel.remove(best_candidate)
                    inmodel.append(best_candidate)
                    current_score = best_new_score
                    stop=0
                    forward = forward+1
                else:
                    print("falis forward")
                    stop=stop+1
            else:
                for candidate in inmodel:
                    temp_var = inmodel.copy()
                    temp_var.remove(candidate)
                    Temp_X_train = X_train.iloc[:, temp_var]
                    Temp_X_val   = X_val.iloc[:, temp_var]
                    model.fit(Temp_X_train,y_train)
                    prediction = model.predict(Temp_X_val)
                    score = Acc(prediction>=threshold, y_val, pri=False)
                    score_candidate.append((score, candidate))
                
                score_candidate.sort()
                best_new_score, best_candidate = score_candidate.pop()
                    
                if current_score<best_new_score:
                    print("success backward")
                    outmodel.append(best_candidate)
                    inmodel.remove(best_candidate)
                    current_score = best_new_score
                    stop=0
                    backward = backward+1
                else:
                    print("fails backward")
                    stop=stop+1
                
            print(inmodel)
            
        Temp_X_train = X_train.iloc[:, inmodel]
        Temp_X_val   = X_val.iloc[:, inmodel]
        model.fit(Temp_X_train, y_train)
        prediction = model.predict(Temp_X_val)
        score = Acc(prediction>=threshold, y_val, pri=False)
        print("Best Validation Acc:", round((score)*100, 1), "%")
        print("선택된 변수의 개수:", len(inmodel), "개")
        print("success forward:", forward)
        print("success backward:", backward)
        
    return model, inmodel

In [ ]:
#Example
model, inmodel = variable_selection(LogisticRegression(solver="lbfgs",max_iter=300), 
                                    X_train, y_train, X_val, y_val, direction="forward")

### STEP 8. Sickit-learn 패키지 사용

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import accuracy_score

def variable_selection_scikit(model, X_train, y_train, X_val, y_val, direction="forward", n_features_to_select=None, threshold=0.5):
    # Define the feature selector
    feature_selector = SequentialFeatureSelector(model, direction=direction, n_features_to_select=n_features_to_select)

    # Fit the model
    feature_selector.fit(X_train, y_train)

    # Get the selected features
    selected_features = X_train.columns[feature_selector.get_support()]
    print([X_train.columns.get_loc(c) for c in selected_features if c in X_train])

    Temp_X_train = X_train[selected_features]
    Temp_X_val   = X_val[selected_features]
                                        
    model.fit(Temp_X_train, y_train)
    prediction = model.predict(Temp_X_val)
    score = Acc(prediction>=threshold, y_val, pri=False)
    print("Best Validation Acc:", round(score*100,1), "%")
    print("선택된 변수의 개수:", len(inmodel), "개")

    return model, selected_features


model, selected_features = variable_selection_scikit(LinearRegression(), X_train, y_train, X_val, y_val, direction="forward",n_features_to_select=8, threshold=0.5)

---